In [1]:
from xgboost.sklearn import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
class client():
    def __init__(self,data,cid):
        self.data = data
        self.model = XGBClassifier(random_state=22)
        self.cid = cid

        
    def update(self,best_model):
        #最好的作为模型
        self.model = best_model
        x = self.data.iloc[:,1:]
        y = self.data["label"]
        x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=2)
        
        self.model =XGBClassifier().fit(x_train,y_train,xgb_model = best_model)
        
        
    def fit(self):
        params={
            'max_depth':range(2,30,2),
            'n_estimators':range(100,500,200),
            'learning_rate':[0.05,0.1,0.3,0.4,0.5],
            'subsample':np.array([0.7,0.8,0.9]),
            "min_child_weight": [1,2,3,4]
            
            }
        
       
        x = self.data.iloc[:,1:]
        y = self.data["label"]
        
        x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=2)
        
        CV_model = GridSearchCV(XGBClassifier(), param_grid=params, n_jobs= -1,scoring='roc_auc', verbose=10)
            
        self.model = CV_model.fit(x_train,y_train).best_estimator_

In [3]:
class sever():
    def __init(self):
        pass
    def fuse(self):
        pass

In [4]:
def cross_10(model_clt,DB):
    auc_score_10 = []
    recall_score_10 = []
    acc_score_10 = []
    best_auc = 0
    for data in DB: 
        
        x_train,x_test,y_train,y_test = train_test_split(data.iloc[:,1:],data["label"],test_size=0.3,random_state=2)
        
        y_pred = model_clt.predict(x_test)
        
        recall = recall_score(y_test,y_pred)
        acc = accuracy_score(y_test,y_pred)
        auc = roc_auc_score(y_test,model_clt.predict_proba(x_test)[:, 1])
        
#         print(cross_val_score(model,client_data[i].iloc[:,1:],client_data[i].iloc[:,0],cv = 5,scoring = "roc_auc"))
        
     
        
        if best_auc < auc:
            best_auc = auc
        auc_score_10.append(auc)
        recall_score_10.append(recall)
        acc_score_10.append(acc)
   
    return np.array(auc_score_10).mean(),best_auc,auc_score_10,recall_score_10,acc_score_10
def find_max_index(score_list):
    index = 0
    for i in range(len(score_list)):
    
        if score_list[i]==max(score_list):
            index = i
    return index

In [5]:
feature_3 = ["label","COM8","COM10","COM11","hospitalid"]
feature_7 = ["label","COM1","COM3","COM4","COM8","COM10","COM11","LAB15","hospitalid"]
feature_13 = ["label","COM1","COM3","COM4","COM7","COM8","COM10","COM11","LAB4","LAB15","LAB25","LAB26","LAB32","SBP","hospitalid"]

In [6]:
# DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/Imp6_cart_2.0.csv").drop(columns=["Unnamed: 0","Unnamed: 0.1"],axis = 1)
DB_1 = pd.read_csv("D:/实验室/数据/eicu数据处理/tenSiteDTDisc.csv").drop(columns=["Unnamed: 0","patientunitstayid","creatinine"],axis = 1)
DB_1 = DB_1.sample(frac=1)

In [7]:

DB_1["label"].value_counts()
DB_1 = DB_1.astype(np.int16)
DB_1 = DB_1.loc[:,feature_3]
# 数据分割
hos_id = [122, 141, 142, 227, 243, 382, 390, 403, 420, 435]
client_data = []
for hosid in hos_id:
    client_data.append(DB_1[DB_1["hospitalid"]==hosid ].iloc[:,:].drop(columns = ["hospitalid"],axis = 1))

feature = client_data[0].columns

In [8]:
DB_name = []
BN_name = []
for i in range(1,11):
    DB_name.append("data_"+str(i))
    BN_name.append("CL"+str(i))
BN_name.append("Fuse")    

In [9]:
best_id = 0
best_auc = 0
best_mode = 0
for i in range(10):
    l1= []
    model = XGBClassifier()
    x_train,x_test,y_train,y_test = train_test_split(client_data[i].iloc[:,1:],client_data[i]["label"],test_size=0.3,random_state=2)
    model = model.fit(x_train,y_train)
    for j in range(10):
        x_train,x_test,y_train,y_test = train_test_split(client_data[j].iloc[:,1:],client_data[j]["label"],test_size=0.3,random_state=2)
        y_pred = model.predict(x_test)
        auc = roc_auc_score(y_test,model.predict_proba(x_test)[:, 1])
        l1.append(auc)
    l1 = np.array(l1)
    print(l1.mean())
    
    if l1.mean()>best_auc:
        best_auc = l1.mean() 
        best_id = i
        best_mode = model
        


0.8565314550442762
0.8460350649009449
0.8551822848841466
0.8342786929716872
0.8348286704176318
0.8198899075418928
0.8349898964986524
0.8132816529176614
0.8639045174060331
0.8387379150806007


In [10]:
best_id = 0
best_auc = 0
best_model = 0
for i in range(10):
    l1= []
    model = best_model
    x_train,x_test,y_train,y_test = train_test_split(client_data[i].iloc[:,1:],client_data[i]["label"],test_size=0.3,random_state=2)
    model = model.fit(x_train,y_train,xgb_model = best_model)
    
    
    for j in range(10):
        x_train,x_test,y_train,y_test = train_test_split(client_data[j].iloc[:,1:],client_data[j]["label"],test_size=0.3,random_state=2)
        y_pred = model.predict(x_test)
        auc = roc_auc_score(y_test,model.predict_proba(x_test)[:, 1])
        l1.append(auc)
    l1 = np.array(l1)
    print(l1.mean())
    
    if l1.mean()>best_auc:
        best_auc = l1.mean() 
        best_id = i
        best_mode = model
        

AttributeError: 'int' object has no attribute 'fit'

In [9]:
model = XGBClassifier()
cross_val_score(model,client_data[4].iloc[:,1:],client_data[4]["label"],scoring="roc_auc")

array([0.71423599, 0.73073512, 0.73783712, 0.71669234, 0.6854578 ])

In [10]:
model = XGBClassifier()
x_train,x_test,y_train,y_test = train_test_split(client_data[4].iloc[:,1:],client_data[4]["label"],test_size=0.3,random_state=2)
model = model.fit(x_train,y_train)
for j in range(10):
    x_train,x_test,y_train,y_test = train_test_split(client_data[j].iloc[:,1:],client_data[j]["label"],test_size=0.3,random_state=2)
    auc = roc_auc_score(y_test,model.predict_proba(x_test)[:, 1])
    print(auc)

0.7921704578071183
0.7797915392901854
0.7913463395789067
0.7156800242865817
0.7162687069110476
0.6807500778089013
0.7444090562902443
0.7098629466470876
0.8341734815821539
0.7909449307754393


In [11]:
len(client_data[0].columns)

4

In [12]:

client_num= 10
clients = []
for i in range(client_num):
# 创建客户端，并且传入数据和结构学习模型
    clients.append(client(client_data[i],i))
round_num = 0
auc_score_list = []
recall_score_list = []
acc_score_list = []
while(round_num<10):
    #开始训练
    if round_num==0:
        print("开始初始训练")
        for clt in clients:
            clt.fit()
    max_id = []
    #记录最好的模型
    auc_score_list = []
    best_model = None
    best_client = 0
    mean_auc_list = []
    best_auc_list = []
    #记录评分
    for clt in clients:
        
        auc_mean,best_auc,auc,recall,acc= cross_10(clt.model,client_data)
        auc_score_list.append(auc)
        mean_auc_list.append(auc_mean)
        best_auc_list.append(best_auc)
        
    print(best_auc_list)
    best_id = find_max_index(best_auc_list)
    best_model = clients[best_id].model
    print(best_id)

    #更新
    for clt in clients:
        clt.update(best_model)
        
    round_num+=1
    
    
    
    
auc_score_list = []
recall_score_list = []
acc_score_list = []

for clt in clients:
    auc_mead,best_auc,auc,recall,acc= cross_10(clt.model,client_data)   
    auc_score_list.append(auc)
    mean_auc_list.append(auc_mead)
    best_auc_list.append(best_auc)
    
    recall_score_list.append(recall)
    acc_score_list.append(acc)
    
    
    
    
    
auc_mead,best,auc,recall,acc= cross_10(best_model,client_data)    
auc_score_list.append(auc)
recall_score_list.append(recall)
acc_score_list.append(acc)


BN_name = np.array(BN_name).T
auc_score_list = np.array(auc_score_list).T
recall_score_list = np.array(recall_score_list).T
acc_score_list = np.array(acc_score_list).T
auc_result = pd.DataFrame(auc_score_list,index = hos_id,columns=BN_name).round(4)   
recall_result = pd.DataFrame(recall_score_list,index = hos_id,columns=BN_name).round(4)
acc_result = pd.DataFrame(acc_score_list,index = hos_id,columns=BN_name).round(4)

开始初始训练
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
Fitting 5 folds for each of 1680 candidates, totalling 8400 fits
[0.8304360179023338, 0.8305414635860904, 0.8365518675602212, 0.8296246719467616, 0.8319005412878433, 0.829510439122692, 0.825477141718999, 0.8319005412878433, 0.8360070648608117, 0.8340211711500609]
2
[0.8274132416346425, 0.8293346963164308, 0.8363761247539601, 0.828130858093542, 0.8331688185396944, 0.8310452596307059, 0.8256118778704657, 0.

In [17]:
auc_result

,CL1,CL2,CL3,CL4,CL5,CL6,CL7,CL8,CL9,CL10,Fuse
122,0.7700,0.8076,0.7917,0.7932,0.7975,0.7891,0.7945,0.7801,0.7935,0.8057,0.7914
141,0.7897,0.7896,0.7806,0.7801,0.7767,0.8001,0.7750,0.7865,0.7832,0.7535,0.7819
142,0.7743,0.7785,0.7960,0.7730,0.7901,0.7800,0.7619,0.7766,0.7938,0.7845,0.7952
227,0.6950,0.7265,0.7049,0.7299,0.7183,0.7079,0.7280,0.7003,0.7020,0.7128,0.7062
243,0.7200,0.7258,0.7216,0.7232,0.7167,0.7196,0.7184,0.7242,0.7213,0.7254,0.7208
382,0.6726,0.6736,0.6749,0.6668,0.6824,0.6727,0.6587,0.6770,0.6770,0.6793,0.6748
390,0.7351,0.7283,0.7364,0.7369,0.7434,0.7363,0.7371,0.7283,0.7346,0.7319,0.7373
403,0.7061,0.7056,0.7105,0.7086,0.7100,0.7026,0.7007,0.7076,0.7060,0.7078,0.7100
420,0.8271,0.8313,0.8366,0.8299,0.8339,0.8295,0.8284,0.8295,0.8360,0.8339,0.8362
435,0.8017,0.7910,0.7980,0.7893,0.7903,0.7918,0.7883,0.8026,0.7971,0.7990,0.7956


In [23]:
recall_result

,CL1,CL2,CL3,CL4,CL5,CL6,CL7,CL8,CL9,CL10,Fuse
122,0.7981,0.7981,0.7981,0.7981,0.0,0.7981,0.7981,0.0,0.7981,0.7981,0.7981
141,0.7757,0.7757,0.7757,0.7757,0.0,0.7757,0.7757,0.0,0.7757,0.7757,0.7757
142,0.7161,0.7161,0.7161,0.7161,0.0,0.7161,0.7161,0.0,0.7161,0.7161,0.7161
227,0.7778,0.7778,0.7778,0.7778,0.0,0.7778,0.7778,0.0,0.7778,0.7778,0.7778
243,0.8339,0.8339,0.8339,0.8339,0.0,0.8339,0.8339,0.0,0.8339,0.8339,0.8339
382,0.7745,0.7745,0.7745,0.7745,0.0,0.7745,0.7745,0.0,0.7745,0.7745,0.7745
390,0.8252,0.8252,0.8252,0.8252,0.0,0.8252,0.8252,0.0,0.8252,0.8252,0.8252
403,0.8222,0.8222,0.8222,0.8222,0.0,0.8222,0.8222,0.0,0.8222,0.8222,0.8222
420,0.7713,0.7713,0.7713,0.7713,0.0,0.7713,0.7713,0.0,0.7713,0.7713,0.7713
435,0.8814,0.8814,0.8814,0.8814,0.0,0.8814,0.8814,0.0,0.8814,0.8814,0.8814


In [22]:
acc_result

,CL1,CL2,CL3,CL4,CL5,CL6,CL7,CL8,CL9,CL10,Fuse
122,0.7823,0.7823,0.7823,0.7823,0.7204,0.7823,0.7823,0.7204,0.7823,0.7823,0.7823
141,0.7593,0.7593,0.7593,0.7593,0.7500,0.7593,0.7593,0.7500,0.7593,0.7593,0.7593
142,0.7832,0.7832,0.7832,0.7832,0.6830,0.7832,0.7832,0.6830,0.7832,0.7832,0.7832
227,0.6804,0.6804,0.6804,0.6804,0.7385,0.6804,0.6804,0.7385,0.6804,0.6804,0.6804
243,0.6636,0.6636,0.6636,0.6636,0.7083,0.6636,0.6636,0.7083,0.6636,0.6636,0.6636
382,0.6299,0.6299,0.6299,0.6299,0.7119,0.6299,0.6299,0.7119,0.6299,0.6299,0.6299
390,0.7275,0.7275,0.7275,0.7275,0.5855,0.7275,0.7275,0.5855,0.7275,0.7275,0.7275
403,0.6435,0.6435,0.6435,0.6435,0.7161,0.6435,0.6435,0.7161,0.6435,0.6435,0.6435
420,0.8512,0.8512,0.8512,0.8512,0.7072,0.8512,0.8512,0.7072,0.8512,0.8512,0.8512
435,0.7570,0.7570,0.7570,0.7570,0.6982,0.7570,0.7570,0.6982,0.7570,0.7570,0.7570


In [24]:
recall_result.to_csv("D:/实验室/图片/FD/result.csv")